In [6]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 341 kB in 2s (175 kB/s)
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
chromium-chromedriver is already the newest version (1:85.0.4183.83-0

In [7]:
from selenium import webdriver # 웹 브라우저 자동화
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
from urllib.parse import quote_plus
from selenium.webdriver.common.keys import Keys
import time # 시간 지연
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

In [45]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

url = 'https://www.melon.com/chart/week/index.htm?classCd=GN0700'
driver.get(url)
html = driver.page_source
soup = bs(html, 'html.parser')
songs = soup.select('.lst50, .lst100')

In [46]:
data_list = []

In [47]:
for song in songs:
    # 차트 페이지에서의 정보 수집
    title = song.select_one('.ellipsis.rank01 a').text
    rank = song.select_one('.rank').text
    singer = song.select_one('.ellipsis.rank02 a').text
    album = song.select_one('.ellipsis.rank03 a').text
    songid = song.select_one('.like')['data-song-no']

    # 상세 페이지로 이동
    BASE_DETAIL_URL = "https://www.melon.com/song/detail.htm?songId="
    detail_url = BASE_DETAIL_URL + songid
    driver.get(detail_url)
    detail_soup = bs(driver.page_source, 'html.parser')

    # 상세 페이지에서의 정보 수집
    genre = detail_soup.select_one('.meta .list dd').find_next_sibling('dd').find_next_sibling('dd').text.strip()
    release_date = detail_soup.select_one('.meta .list dd').find_next_sibling('dd').text.strip()
    album_image_url = detail_soup.select_one('.thumb .image_typeAll img')['src']
    likes = detail_soup.select_one('#d_like_count').text.strip()
    comments_count = detail_soup.select_one('#revCnt').text.strip()
    lyrics = detail_soup.select_one('.lyric').text.strip()
    composers = []
    lyricists = []
    arrangers = []

    # 각 li 항목을 반복 처리
    for li in detail_soup.select('.list_person.clfix li'):
      artist_name_el = li.select_one('.ellipsis .artist_name')
      artist_type_el = li.select_one('.meta .type')

      if artist_name_el and artist_type_el:  # 두 항목이 모두 존재할 때만 처리
        artist_name = artist_name_el.text.strip()
        artist_type = artist_type_el.text.strip()

        if artist_type == "작곡":
            composers.append(artist_name)
        elif artist_type == "작사":
            lyricists.append(artist_name)
        elif artist_type == "편곡":
            arrangers.append(artist_name)
    print("Composers:", composers)
    print("Lyricists:", lyricists)
    print("Arrangers:", arrangers)

    data_list.append([songid, title, rank, singer, album, likes, release_date, genre, comments_count, lyrics, album_image_url, ", ".join(composers), ", ".join(lyricists), ", ".join(arrangers)])

Composers: ['조영수']
Lyricists: ['김이나']
Arrangers: ['조영수', '이유진']
Composers: ['박상철']
Lyricists: ['박진복']
Arrangers: ['이창우 (날아라야옹이)']
Composers: ['영탁', '지광민']
Lyricists: ['영탁', '지광민']
Arrangers: ['영탁', '지광민']
Composers: ['알고보니 혼수상태']
Lyricists: ['알고보니 혼수상태']
Arrangers: ['알고보니 혼수상태']
Composers: ['영탁', '지광민']
Lyricists: ['영탁', '지광민']
Arrangers: ['영탁', '지광민']
Composers: ['알고보니 혼수상태', '김지환']
Lyricists: ['알고보니 혼수상태', '사마천']
Arrangers: ['알고보니 혼수상태', '이지수']
Composers: ['구희상', '지광민', '영탁']
Lyricists: ['구희상', '지광민', '영탁']
Arrangers: ['구희상', '지광민', '영탁']
Composers: ['김지환', '알고보니 혼수상태']
Lyricists: ['김지환', '알고보니 혼수상태']
Arrangers: ['김지환', '알고보니 혼수상태', '진실이']
Composers: ['설운도']
Lyricists: ['설운도']
Arrangers: ['송기영']
Composers: ['김지환', '알고보니 혼수상태']
Lyricists: ['김지환', '알고보니 혼수상태']
Arrangers: ['김지환', '알고보니 혼수상태']
Composers: ['송결']
Lyricists: ['김학진']
Arrangers: ['임현기']
Composers: ['윤일상']
Lyricists: ['이건우', '신철']
Arrangers: ['김창락', '김수빈(AIMING)', '임수혁 (AIMING)']
Composers: ['윤명선']
Lyricists: ['윤명선', 'GLABINGO

In [48]:
# CSV 파일로 저장
import csv
with open('trot_songs.csv', 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["SongID", "Title", "Rank", "Singer", "Album", "Likes", "ReleaseDate", "Genre", "CommentsCount", "Lyrics", "AlbumImageURL", "Composers", "Lyricists", "Arrangers"])
    writer.writerows(data_list)

In [8]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=chrome_options)

url = 'https://www.melon.com/chart/week/index.htm?classCd=GN0800'
driver.get(url)
html = driver.page_source
soup = bs(html, 'html.parser')
songs = soup.select('.lst50, .lst100')

In [9]:
data_list2 = []

In [10]:
for song in songs:
    # 차트 페이지에서의 정보 수집
    title = song.select_one('.ellipsis.rank01 a').text
    rank = song.select_one('.rank').text
    singer = song.select_one('.ellipsis.rank02 a').text
    album = song.select_one('.ellipsis.rank03 a').text
    songid = song.select_one('.like')['data-song-no']

    # 상세 페이지로 이동
    BASE_DETAIL_URL = "https://www.melon.com/song/detail.htm?songId="
    detail_url = BASE_DETAIL_URL + songid
    driver.get(detail_url)
    detail_soup = bs(driver.page_source, 'html.parser')

    # 상세 페이지에서의 정보 수집
    genre = detail_soup.select_one('.meta .list dd').find_next_sibling('dd').find_next_sibling('dd').text.strip()
    release_date = detail_soup.select_one('.meta .list dd').find_next_sibling('dd').text.strip()
    album_image_url = detail_soup.select_one('.thumb .image_typeAll img')['src']
    likes = detail_soup.select_one('#d_like_count').text.strip()
    comments_count = detail_soup.select_one('#revCnt').text.strip()
    lyrics = detail_soup.select_one('.lyric').text.strip()
    composers = []
    lyricists = []
    arrangers = []

    # 각 li 항목을 반복 처리
    for li in detail_soup.select('.list_person.clfix li'):
      artist_name_el = li.select_one('.ellipsis .artist_name')
      artist_type_el = li.select_one('.meta .type')

      if artist_name_el and artist_type_el:  # 두 항목이 모두 존재할 때만 처리
        artist_name = artist_name_el.text.strip()
        artist_type = artist_type_el.text.strip()

        if artist_type == "작곡":
            composers.append(artist_name)
        elif artist_type == "작사":
            lyricists.append(artist_name)
        elif artist_type == "편곡":
            arrangers.append(artist_name)
    print("Composers:", composers)
    print("Lyricists:", lyricists)
    print("Arrangers:", arrangers)

    data_list2.append([songid, title, rank, singer, album, likes, release_date, genre, comments_count, lyrics, album_image_url, ", ".join(composers), ", ".join(lyricists), ", ".join(arrangers)])

Composers: ['이찬혁', '로빈(ROVIN)']
Lyricists: ['이찬혁']
Arrangers: []
Composers: ['Aaron Wagner', 'Levi Hummon', 'Drew Schueler']
Lyricists: ['강은정']
Arrangers: ['Aaron Wagner']
Composers: ['10CM', '고영배']
Lyricists: ['10CM']
Arrangers: ['10CM', '이요한', '성수용', '이윤혁', '인재']
Composers: ['Jisoo Park (153/Joombas)']
Lyricists: ['Jisoo Park (153/Joombas)']
Arrangers: ['Jisoo Park (153/Joombas)']
Composers: ['송봉주 (자전거 탄 풍경)']
Lyricists: ['송봉주 (자전거 탄 풍경)']
Arrangers: ['박용준']
Composers: ['백아']
Lyricists: ['백아']
Arrangers: ['백아']
Composers: ['이찬혁']
Lyricists: ['이찬혁']
Arrangers: ['신승익', '서기']
Composers: ['이찬혁']
Lyricists: ['이찬혁']
Arrangers: ['로빈(ROVIN)', '이찬혁']
Composers: ['10CM']
Lyricists: ['10CM']
Arrangers: ['10CM', '이요한', '성수용', '이윤혁', '방인재']
Composers: ['이찬혁']
Lyricists: ['이찬혁']
Arrangers: ['서기', '신승익']
Composers: ['안지영', '바닐라맨 (바닐라어쿠스틱)']
Lyricists: ['안지영']
Arrangers: ['바닐라맨 (바닐라어쿠스틱)']
Composers: ['Egg1', 'Egg2', 'Egg3']
Lyricists: ['Egg1', 'Egg2', 'Egg3']
Arrangers: ['Egg1', 'Egg2', 'Egg3']
Com

In [12]:
# CSV 파일로 저장
import csv
with open('blues_songs.csv', 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(["SongID", "Title", "Rank", "Singer", "Album", "Likes", "ReleaseDate", "Genre", "CommentsCount", "Lyrics", "AlbumImageURL", "Composers", "Lyricists", "Arrangers"])
    writer.writerows(data_list2)